In [0]:
# -*- coding: utf-8 -*-
#Import the necessary methods from tweepy library
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import json

#Variables that contains the user credentials to access Twitter API
access_token = ""
access_token_secret = ""
consumer_key = ""
consumer_secret = ""

# # # # TWITTER STREAMER # # # #
class TwitterStreamer():
    """
    Class for streaming and processing live tweets.
    """

    def __init__(self):
        pass

    def stream_tweets(self, fetched_tweets_filename, hash_tag_list):
        # This handles Twitter authetification and the connection to Twitter Streaming API
        listener = StdOutListener(fetched_tweets_filename)
        auth = OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token, access_token_secret)
        stream = Stream(auth, listener)

        # This line filter Twitter Streams to capture data by the keywords:
        stream.filter(languages=["en"], track=hash_tag_list)


# # # # TWITTER STREAM LISTENER # # # #
class StdOutListener(StreamListener):
    """
    This is a basic listener that just prints received tweets to stdout.
    """
    count=0

    def __init__(self, fetched_tweets_filename):
        self.fetched_tweets_filename = fetched_tweets_filename

    def on_data(self, data):
        try:
            # print(data)
            with open(self.fetched_tweets_filename, 'a', newline='') as tf:
                tweet = json.dumps(data, ensure_ascii=False)
                tf.write(data)
                if (self.count%100==0):print(self.count)
                self.count=self.count+1

            #		tweet = json.loads(data)
            #      	    with open('your_data.json', 'a') as my_file:
            #                json.dump(tweet, my_file)
            return True
        except BaseException as e:
            print("Error on_data %s" % str(e))
        return True

    def on_error(self, status):
        print("error "+str(status))


if __name__ == '__main__':
    # Authenticate using config.py and connect to Twitter Streaming API.
    hash_tag_list = ['stockmarket', 'bitcoin', 'money', 'trading', 'forextrader', 'investment', 'wallstreet', 'stocks', 'entrepreneur', 'forex', 'trader', 'investor', 'investing', 'cryptocurrency', 'invest', 'business', 'daytrader', 'binaryoptions', 'forexsignals', 'profit', 'success', 'finance', 'wealth', 'makemoneyonline', 'forexlifestyle', 'forextrading', 'motivation', 'millionaire', 'entrepreneurship', 'daytrading']
    fetched_tweets_filename = "tweets7.json"

    twitter_streamer = TwitterStreamer()
    twitter_streamer.stream_tweets(fetched_tweets_filename, hash_tag_list)


In [0]:
!pip install --force-reinstall GetOldTweets3

     |████████████████████████████████| 5.8MB 3.9MB/s 
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [0]:
import GetOldTweets3 as got
from datetime import*
import json,time
from bson import json_util

def save_30_day_json(tweets,search_keyword):
  a = [json.loads(json.dumps(tweets[i].__dict__,  default=json_util.default)) for i in range(len(tweets))]
  with open(search_keyword+'.json', 'a') as outfile:
      json.dump(a, outfile)
  print ('saved successfully in ',search_keyword+'.json','file....')

def get_tweets(start_date,search_keyword,debug=False):
  q = start_date.split('-')
  a = datetime(int(q[0]),int(q[1]),int(q[2]))
  a+=timedelta(days=1)
  end_date = (a.strftime('%Y-%m-%d'))
  tweetCriteria = got.manager.TweetCriteria().setQuerySearch(search_keyword)\
                                            .setLang('en')\
                                            .setSince(start_date)\
                                            .setUntil(end_date)\
                                            .setMaxTweets(3000)
                                            # .setTopTweets(True)\
  tweet_batch = got.manager.TweetManager.getTweets(tweetCriteria)
  if debug: print(start_date,search_keyword,len(tweet_batch))
  return tweet_batch

def get_30days_tweets(start_date,search_keyword,debug=False):
  tweets = []
  for i in range(30):
    tweet_batch = get_tweets(start_date,search_keyword,debug)
    tweets.extend(tweet_batch)
    q = start_date.split('-')
    a = datetime(int(q[0]),int(q[1]),int(q[2]))
    a+=timedelta(days=1)
    start_date = (a.strftime('%Y-%m-%d'))
    open('start_date','w').write(start_date)
    save_30_day_json(tweet_batch,search_keyword)
    # time.sleep(30)
    # if debug: print (len(tweets),start_date,search_keyword)
  return tweets



open('start_date','w').write('2017-10-01')
open('company_list','w').write("AMZN")

4

In [0]:
import time
while(1):
  temp = open('company_list','r').read()
  keyword_list=list(temp.split('-'))
  loop_keyword_list = list(keyword_list)

  start_date =open('start_date','r').read()
  try:
    for search_keyword in loop_keyword_list:
      print ('Running for ', search_keyword.upper())
      tweets = get_30days_tweets(start_date,search_keyword,debug=True)
      print ('Tweets downloaded..')
      keyword_list.remove(search_keyword)
      open('company_list','w').write('-'.join(keyword_list))

      
  except Exception as e:
    print (e)
    open('start_date','w').write(start_date)
    open('company_list','w').write('-'.join(keyword_list))
    print ('error occoured on: ')
    print (start_date,search_keyword)
    time.sleep(30)#sleeping for too many request error
    print ('continuing')
    continue

In [0]:
temp = open('company_list','r').read()
keyword_list=list(temp.split('-'))
loop_keyword_list = list(keyword_list)

start_date =open('start_date','r').read()
print (start_date,loop_keyword_list)

2020-01-01 ['AAPL', 'TSLA', 'GOOGL', 'AMZN']


In [0]:
with open('data.json','r') as out:
  red = json.load(out)

print ((red[0].keys()))
# print ((red[0]['formatted_date']))
# p= datetime.datetime.fromtimestamp(1577918767)
# print (p.strftime('%B/%Y/%d/ %H-%M-%S'))

for i in red:
  if i['id'] == red[0]['id']:
    print (i['text'])